Скопируем код из метолдички для первого задания

In [1]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np


In [2]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_features = 2, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, flip_y =0.12, n_samples = 1100,random_state=5)

In [3]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [4]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [5]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction      

In [6]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [33]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [38]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [39]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels,type_qualitu = 'gini'):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5
    if type_qualitu == 'entropy':
        current_qualitu = entropy(labels)
    else:
        current_qualitu = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_qualitu,type_qualitu=type_qualitu)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [40]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels,type_qualitu='gini'):

    quality, t, index = find_best_split(data, labels,type_qualitu=type_qualitu)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels,type_qualitu=type_qualitu)
    false_branch = build_tree(false_data, false_labels,type_qualitu=type_qualitu)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [11]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [12]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [13]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    


In [14]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).

Перепишем функцию build_tree с ограничением на глубину дерева.
build_tree_limit

In [50]:
def build_tree_limit(data, labels,limit,type_qualitu='gini'):
    limit-=1
    quality, t, index = find_best_split(data, labels,type_qualitu=type_qualitu)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0 or limit == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree_limit(true_data, true_labels,limit,type_qualitu=type_qualitu)
    false_branch = build_tree_limit(false_data, false_labels,limit,type_qualitu=type_qualitu)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [78]:
my_tree_limit = build_tree_limit(train_data, train_labels,5)

In [79]:
my_tree = build_tree(train_data, train_labels)

In [80]:
# подсчитаем максимальную глубину дерева
def depth_tree(node,n=1):

    # Если лист, то выводим глубину долиста
    if isinstance(node, Leaf):   
        return n
    
    # Рекурсионный вызов функции на положительном поддереве
    l = depth_tree(node.true_branch, n=n+1)

    # Рекурсионный вызов функции на положительном поддереве
    m =  depth_tree(node.false_branch, n=n+1)
    return l,m

In [81]:
a,b = depth_tree(my_tree_limit)
a,b

((((5, 5), (5, 5)), 3), ((4, 4), ((5, 5), (5, 5))))

In [82]:
a,b = depth_tree(my_tree)
a,b

((((5,
    ((((9, (10, (11, 11))), 8), (8, 8)),
     (((9, 9), 8),
      (8,
       (((((13, 13), 12), (12, (13, 13))), ((12, 12), 11)),
        ((11, 11), 10)))))),
   (5, ((7, 7), 6))),
  3),
 ((4, 4),
  ((((7, (8, 8)), 6), (6, 6)),
   (((((9, (10, (11, 11))), 8), (8, 8)), 6), ((7, (8, 8)), 6)))))

In [83]:
# Изучим качество  ответов для обучающей и тестовой выборки по дереву без ограничения
train_answers = predict(train_data, my_tree)
answers = predict(test_data, my_tree)
train_accuracy = tr_nolimit_dgini =accuracy_metric(train_labels, train_answers)
test_accuracy = test_nolimit_dgini=accuracy_metric(test_labels, answers)
print (f'Дерево без ограничений критерий джини, критерий Джини: Качество на обучаеющей выборке {train_accuracy}, качество на тесовой выборке {test_accuracy}')

Дерево без ограничений критерий джини, критерий Джини: Качество на обучаеющей выборке 93.24675324675324, качество на тесовой выборке 91.81818181818183


In [84]:
# Изучим качество  ответов для обучающей и тестовой выборки по дереву с ограничением
train_answers = predict(train_data, my_tree_limit)
answers = predict(test_data, my_tree_limit)
train_accuracy = tr_limit_dgini=accuracy_metric(train_labels, train_answers)
test_accuracy = test_limit_dgini =accuracy_metric(test_labels, answers)
print (f'Дерево c ограниченим критерий джини: Качество на обучаеющей выборке {train_accuracy}, качество на тесовой выборке {test_accuracy}')

Дерево c ограниченим критерий джини: Качество на обучаеющей выборке 93.24675324675324, качество на тесовой выборке 92.42424242424242


Как видно качество ответов на тестоваой выборке лучше у дерева с ограничением глубины

2. Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы.

In [85]:
# Расчет критерия Энтропии

def entropy(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 0
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p *np.log2(p)
        
    return impurity

In [86]:
# Расчет качества добавим аргумент, тип оценки

def quality(left_labels, right_labels, current_quality,type_qualitu = 'gini'):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    if type_qualitu == 'entropy':
        return current_quality - p * entropy(left_labels) - (1 - p) * entropy(right_labels)  
    return current_quality - p * gini(left_labels) - (1 - p) * gini(right_labels)

добавим данный аргумент в другие зависимые функции

In [87]:
# Построим дерево с криетерием энтропии
my_tree = build_tree(train_data, train_labels,type_qualitu = 'entropy')

In [88]:
# Изучим качество  ответов для обучающей и тестовой выборки по дереву без ограничения c оценкой энтропия
train_answers = predict(train_data, my_tree)
answers = predict(test_data, my_tree)
train_accuracy = tr_nolimit_entropy=accuracy_metric(train_labels, train_answers)
test_accuracy =test_nolimit_entropy= accuracy_metric(test_labels, answers)
print (f'Дерево без ограничений, критерий энтропия: Качество на обучаеющей выборке {train_accuracy}, качество на тесовой выборке {test_accuracy}')

Дерево без ограничений, критерий энтропия: Качество на обучаеющей выборке 93.24675324675324, качество на тесовой выборке 92.42424242424242


In [89]:
# Построим дерево с критерием Энтропия с лимитом
my_tree_limit = build_tree_limit(train_data, train_labels,5,type_qualitu = 'entropy')

In [90]:
# Изучим качество  ответов для обучающей и тестовой выборки по дереву с ограничением по критерю энтропия
train_answers = predict(train_data, my_tree_limit)
answers = predict(test_data, my_tree_limit)
train_accuracy = tr_limit_entropy = accuracy_metric(train_labels, train_answers)
test_accuracy = test_limit_entropy = accuracy_metric(test_labels, answers)
print (f'Дерево c ограниченим по критерю энтропия: Качество на обучаеющей выборке {train_accuracy}, качество на тесовой выборке {test_accuracy}')

Дерево c ограниченим по критерю энтропия: Качество на обучаеющей выборке 93.24675324675324, качество на тесовой выборке 92.42424242424242


In [91]:
result_test = np.array([['критерий','Без ограничений','С ограничением'],['джини',test_nolimit_dgini,test_limit_dgini],
                        ['Энтропия',test_nolimit_entropy,test_limit_entropy]])

In [92]:
result_test

array([['критерий', 'Без ограничений', 'С ограничением'],
       ['джини', '91.81818181818183', '92.42424242424242'],
       ['Энтропия', '92.42424242424242', '92.42424242424242']],
      dtype='<U17')

In [93]:
result_train = np.array([['критерий','Без ограничений','С ограничением'],['джини',tr_nolimit_dgini,tr_limit_dgini],
                        ['Энтропия',tr_nolimit_entropy,tr_limit_entropy]])

In [94]:
result_train

array([['критерий', 'Без ограничений', 'С ограничением'],
       ['джини', '93.24675324675324', '93.24675324675324'],
       ['Энтропия', '93.24675324675324', '93.24675324675324']],
      dtype='<U17')

Энтропия дает такие же результаты на тесте, как джини с ограничениме глубины дерева, при этом в энтропии уменьшение глубины не влияет на результат